# Download L3 time-series data from NASA's sensors

**Last updated: 28/04/2024**

[**NASA's ocean color website (OB.DAAC)**](https://oceancolor.gsfc.nasa.gov/data/find-data/) distributes **L1 to L4** data from the most popular satellite sensors, including those not operated by NASA (e.g., ESA's Sentinel-3 OLCI). This script will download L3 time-series data products for our area of study the following sensors: **Aqua-MODIS**, **MERIS**, **SNPP-VIIRS** and **NOAA20-VIIRS** (a.k.a., JPSS1-VIIRS). This script uses the conda environment `bashenv` since the Bash command `wget` is essential for downloading data from OB.DAAC.

Before running the code below, obtain the URLs for your OC-CCI products of interest by following these steps:
1. Visit OB.DAAC Level 3 & 4 Browser portal [here](https://oceancolor.gsfc.nasa.gov/l3/). Select the tab *Extract or Download Data*. 
2. Choose *Mapped* type, *Daily* period, *4 km* resolution and *Extract* for data retrieval method. 
3. Select appropiate coordinates for your area of study. Since the data will not be subset at the exact boundaries provided by your lat/lon (due to precision limits), expand the lat/lon to ensure complete coverage with the subsetting tool (e.g., my area of study, the Endurance site, has the coordinates longitude = [0.265, 1.801] and latitude = [53.758, 54.656], which I have expanded to longitude = [0.20, 1.85] and latitude = [53.70, 54.70]).
4. After product characteristics selection, click *Review Order* and, at the bottom of the Order Summary page, click *Extract*.
5. An email will be sent to you after registration. Reply to the email by sending an empty email. 
6. Wait some time until the data order has been processed, could be minutes up to a few hours, you will eventually receive a confirmation email. The email contains a so-called manifest file that you will use to download the data, as explained [here](https://oceancolor.gsfc.nasa.gov/data/download_methods/?tab=retrieving-orders-tab). 
7. Store the `http_manifest.txt` file inside each corresponding data directory ("DATA_SUBDIRS" in the code below) and run the following Bash code.

In [8]:
%%bash

# Define the name of the manifest file
MANIFEST_FILENAME="http_manifest.txt"

# Define the download directory
ROOT_DIR="../.."  # two directories up
DATA_DIR="data/raw/NASA_data"
DATA_SUBDIRS=(
    "data_timeseries_areastudy_NASA_aquamodis_nc"
    "data_timeseries_areastudy_NASA_meris_nc"
    "data_timeseries_areastudy_NASA_viirssnpp_nc"
    "data_timeseries_areastudy_NASA_viirsjpss1_nc"    
)

# Loop over each subdirectory in the DATA_SUBDIRS array
output_directory_list_path=()
manifest_paths=()
for data_subdir in "${DATA_SUBDIRS[@]}"; do
    # Use printf to create a portable file path
    data_subdir_path=$(printf '%s/%s/%s/' "$ROOT_DIR" "$DATA_DIR" "$data_subdir")
    # Create the directory at the specified path if it doesn't already exist
    if [ ! -d "${data_subdir_path}" ]; then
        mkdir -p "${data_subdir_path}"
        echo "Directory created: ${data_subdir_path}"
    fi
    # Combine data_subdir_path with the MANIFEST_FILENAME and add to manifest_paths
    output_directory_list_path+=("${data_subdir_path}")
    manifest_paths+=("${data_subdir_path}${MANIFEST_FILENAME}")
done 

# Combine manifest_paths and output_directory_list_path using paste with a space delimiter. We will pass
# two arguments at a time (-n 2) to wget, and execute at most 1 parallel wget processes at a time 
# (-P 1). The -q argument executes wget quietly (no output to the terminal) and xarg returns only
# after the last spawned process has finished.
paste -d ' ' <(printf "%s\n" "${manifest_paths[@]}") <(printf "%s\n" "${output_directory_list_path[@]}") | \
xargs -n 2 -P 1 bash -c 'echo "Manifest file path: $1"; echo "Directory path: $2"; wget -q --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --auth-no-challenge=on --no-check-certificate --content-disposition -i "$1" -P "$2"' --

echo "Download completed!"

Directory created: ../../data/raw/NASA_data2/data_timeseries_areastudy_NASA_aquamodis_nc/
Directory created: ../../data/raw/NASA_data2/data_timeseries_areastudy_NASA_meris_nc/
Directory created: ../../data/raw/NASA_data2/data_timeseries_areastudy_NASA_viirssnpp_nc/
Directory created: ../../data/raw/NASA_data2/data_timeseries_areastudy_NASA_viirsjpss1_nc/
Manifest file path: ../../data/raw/NASA_data2/data_timeseries_areastudy_NASA_aquamodis_nc/http_manifest.txt
Directory path: ../../data/raw/NASA_data2/data_timeseries_areastudy_NASA_aquamodis_nc/
Manifest file path: ../../data/raw/NASA_data2/data_timeseries_areastudy_NASA_meris_nc/http_manifest.txt
Directory path: ../../data/raw/NASA_data2/data_timeseries_areastudy_NASA_meris_nc/
Manifest file path: ../../data/raw/NASA_data2/data_timeseries_areastudy_NASA_viirssnpp_nc/http_manifest.txt
Directory path: ../../data/raw/NASA_data2/data_timeseries_areastudy_NASA_viirssnpp_nc/
Manifest file path: ../../data/raw/NASA_data2/data_timeseries_area